# Setup

In [1]:
# Google Only
from google.colab import drive
drive.mount('/content/drive')
ROOT_FOLDER = '/content/drive/My Drive/Code/autocomplete_me/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
## Set Variables for Local and Cloud File Finding
import os
import sys
sys.path.append(ROOT_FOLDER)

In [3]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 27
-rw------- 1 root root 2516 Jul 14 22:20 predict_utils.py
drwx------ 2 root root 4096 Jul 11 13:20 __pycache__
-rw------- 1 root root 3340 Jul 15 12:47 reader.py
-rw------- 1 root root 3580 Jul 14 22:20 train_model_baseline.py
-rw------- 1 root root 3203 Jul 14 22:20 train_utils.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [4]:
from src import utils, reader, predict_utils, train_utils
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)
reload(train_utils)

Using TensorFlow backend.


<module 'src.train_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/train_utils.py'>

## Load Text Data

In [5]:
text = reader.read_bbc_tech()
content_type = 'BBC-TECH'

In [ ]:
text[0]

'US duo in first spam conviction\n\nA brother and sister in the US have been convicted of sending hundreds of thousands of unsolicited e-mail messages to AOL subscribers.\n\nIt is the first criminal prosecution of internet spam distributors. Jurors in Virginia recommended that the man, Jeremy Jaynes, serve nine years in prison and that his sister, Jessica DeGroot, be fined $7,500. They were convicted under a state law that bars the sending of bulk e-mails using fake addresses.\n\nThey will be formally sentenced next year. A third defendant, Richard Rutkowski, was acquitted. Prosecutors said Jaynes was "a snake oil salesman in a new format", using the internet to peddle useless wares, news agency Associated Press reported. A "Fed-Ex refund processor" was supposed to allow people to earn $75 an hour working from home. Another item on sale was an "internet history eraser". His sister helped him process credit card payments. Jaynes amassed a fortune of $24m from his sales, prosecutors said

# Modelling

## Process Text Data

In [6]:
sequences, num_words, word_idx, idx_word = train_utils.preprocess_text(text)

In [ ]:
features, labels = train_utils.pass_sliding_window(sequences, sequence_len=10)

There are 200287 sequences.


In [ ]:
labels = train_utils.one_hot_labels_and_improve_efficiency(labels)

Labels matrix shape:  (200287, 12676)
Labels matrix shape:  (200287, 12676)


In [ ]:
# # Train Tokenizer and Apply
# from tensorflow.keras.preprocessing.text import Tokenizer

# # Train
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(text)

# # Apply Tokenzier on Documents (convert words to numbers)
# sequences = tokenizer.texts_to_sequences(text)

# word_idx = tokenizer.word_index
# num_words = len(word_idx) + 1

In [ ]:
# # Create Sliding Window
# sequence_len = 10

# features = []
# labels = []

# for sequence in sequences:
#     for i in range(len(sequence) - sequence_len):
#         window = sequence[i:sequence_len + i + 1]
        
#         features.append(window[:-1])
#         labels.append(window[-1])

# print(f'There are {len(features)} sequences.')

There are 200287 sequences.


In [ ]:
# # One Hot Encode Labels
# from tensorflow.keras.utils import to_categorical
# labels = to_categorical(labels)
# print('Labels matrix shape: ', labels.shape)

Labels matrix shape:  (200287, 12676)


In [ ]:
# # Convert from float to int8
# from scipy.sparse import csr_matrix
# labels = csr_matrix(labels).astype(np.int8)
# labels = labels.toarray()

In [ ]:
# Create Test Train Set
from sklearn.model_selection import train_test_split

# features = np.array(features)
# labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42, shuffle=True)

In [ ]:
import gc
gc.enable()
del labels
gc.collect()

0

In [ ]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (160229, 10)
X_test shape:  (40058, 10)
y_train shape:  (160229, 12676)
y_test shape:  (40058, 12676)


In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                       y_train:  1.9 GiB
                        y_test: 484.3 MiB
                      features: 15.3 MiB
                       X_train: 12.2 MiB
                        X_test:  3.1 MiB
                      word_idx: 576.1 KiB
                      idx_word: 576.1 KiB
                     sequences:  3.5 KiB
                          text:  3.2 KiB
                            __:  1.7 KiB


In [ ]:
# Embedding Matrix
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 1062 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning:

invalid value encountered in true_divide



array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.02940788,  0.00775488,  0.02958461, ..., -0.0617054 ,
         0.07386386, -0.02477734],
       ...,
       [-0.0030414 ,  0.07376623,  0.15909794, ..., -0.01099743,
        -0.13780413,  0.04205515],
       [-0.02349926, -0.06641477, -0.01852101, ...,  0.07821092,
         0.12694902,  0.1010958 ],
       [ 0.02721107,  0.00886596,  0.24643607, ...,  0.02826212,
         0.23311737,  0.06475616]])

# Design Model

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
# # MODELS_DIR = os.path.join(ROOT_DIR, 'models')
# MODELS_DIR = '/content/drive/My Drive/Code/autocomplete_me/models'
# print(MODELS_DIR)

# DATASET = 'BBC-TECH'

In [ ]:
def train_model(filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=False, epochs=100):
  if not model and not use_pretrained_model:
    print('Provide one of either model or use_pretrained_model.')
  elif model and use_pretrained_model:
      print('Provide one of either model or use_pretrained_model.')
  elif use_pretrained_model:
    model = load_model(model_filepath)
  
  callbacks = [
      EarlyStopping(monitor='val_accuracy', patience=25),
      ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
  ]

  history = model.fit(
      X_train, 
      y_train, 
      epochs=epochs, 
      batch_size=2048, 
      validation_data=(X_test, y_test), 
      verbose=1,
      callbacks=callbacks
  )

  return history

# model_filename = f'{DATASET}-custom-1.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 100

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

##V1

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(64))

model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model_filename = f'{DATASET}-custom-1.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 100

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

/content/drive/My Drive/Code/autocomplete_me/models/BBC-TECH-custom-1.h5
Epoch 1/100
79/79 [==============================] - 7s 90ms/step - loss: 0.7991 - accuracy: 0.7838 - val_loss: 15.4740 - val_accuracy: 0.3160
Epoch 2/100
79/79 [==============================] - 6s 80ms/step - loss: 0.8123 - accuracy: 0.7799 - val_loss: 15.4353 - val_accuracy: 0.3168
Epoch 3/100
79/79 [==============================] - 6s 81ms/step - loss: 0.8044 - accuracy: 0.7830 - val_loss: 15.4022 - val_accuracy: 0.3168
Epoch 4/100
79/79 [==============================] - 6s 81ms/step - loss: 0.8072 - accuracy: 0.7814 - val_loss: 15.3841 - val_accuracy: 0.3173
Epoch 5/100
79/79 [==============================] - 6s 81ms/step - loss: 0.8054 - accuracy: 0.7816 - val_loss: 15.3953 - val_accuracy: 0.3180
Epoch 6/100
79/79 [==============================] - 6s 79ms/step - loss: 0.7968 - accuracy: 0.7840 - val_loss: 15.4555 - val_accuracy: 0.3162
Epoch 7/100
79/79 [==============================] - 6s 80ms/step - l

## V2

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256))

model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model_filename = f'{DATASET}-custom-2.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 500

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

/content/drive/My Drive/Code/autocomplete_me/models/BBC-TECH-custom-2.h5
Epoch 1/500
79/79 [==============================] - 8s 96ms/step - loss: 7.4905 - accuracy: 0.0493 - val_loss: 7.0584 - val_accuracy: 0.0571
Epoch 2/500
79/79 [==============================] - 7s 90ms/step - loss: 6.9629 - accuracy: 0.0578 - val_loss: 6.9854 - val_accuracy: 0.0571
Epoch 3/500
79/79 [==============================] - 8s 101ms/step - loss: 6.8698 - accuracy: 0.0607 - val_loss: 6.9367 - val_accuracy: 0.0673
Epoch 4/500
79/79 [==============================] - 7s 93ms/step - loss: 6.7601 - accuracy: 0.0689 - val_loss: 6.8343 - val_accuracy: 0.0698
Epoch 5/500
79/79 [==============================] - 7s 92ms/step - loss: 6.6199 - accuracy: 0.0782 - val_loss: 6.7341 - val_accuracy: 0.0800
Epoch 6/500
79/79 [==============================] - 7s 93ms/step - loss: 6.4742 - accuracy: 0.0903 - val_loss: 6.6371 - val_accuracy: 0.0937
Epoch 7/500
79/79 [==============================] - 7s 92ms/step - loss: 

## V3

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model_filename = f'{DATASET}-custom-3.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 500

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

/content/drive/My Drive/Code/autocomplete_me/models/BBC-TECH-custom-3.h5
Epoch 1/500
79/79 [==============================] - 7s 95ms/step - loss: 7.4937 - accuracy: 0.0480 - val_loss: 7.0589 - val_accuracy: 0.0571
Epoch 2/500
79/79 [==============================] - 7s 88ms/step - loss: 6.9637 - accuracy: 0.0578 - val_loss: 6.9842 - val_accuracy: 0.0571
Epoch 3/500
79/79 [==============================] - 8s 98ms/step - loss: 6.8810 - accuracy: 0.0588 - val_loss: 6.9417 - val_accuracy: 0.0665
Epoch 4/500
79/79 [==============================] - 7s 90ms/step - loss: 6.7796 - accuracy: 0.0672 - val_loss: 6.8369 - val_accuracy: 0.0706
Epoch 5/500
79/79 [==============================] - 7s 90ms/step - loss: 6.6399 - accuracy: 0.0761 - val_loss: 6.7480 - val_accuracy: 0.0845
Epoch 6/500
79/79 [==============================] - 7s 91ms/step - loss: 6.5218 - accuracy: 0.0874 - val_loss: 6.6856 - val_accuracy: 0.0904
Epoch 7/500
79/79 [==============================] - 7s 90ms/step - loss: 6

## V4

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.2))

model.add(LSTM(64))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model_filename = f'{DATASET}-custom-4.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 500

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

/content/drive/My Drive/Code/autocomplete_me/models/BBC-TECH-custom-4.h5
Epoch 1/500
79/79 [==============================] - 10s 124ms/step - loss: 7.6853 - accuracy: 0.0408 - val_loss: 7.0727 - val_accuracy: 0.0571
Epoch 2/500
79/79 [==============================] - 9s 117ms/step - loss: 7.0305 - accuracy: 0.0578 - val_loss: 7.1292 - val_accuracy: 0.0571
Epoch 3/500
79/79 [==============================] - 9s 116ms/step - loss: 7.0051 - accuracy: 0.0578 - val_loss: 7.1321 - val_accuracy: 0.0571
Epoch 4/500
79/79 [==============================] - 9s 115ms/step - loss: 6.9359 - accuracy: 0.0578 - val_loss: 7.0189 - val_accuracy: 0.0571
Epoch 5/500
79/79 [==============================] - 10s 124ms/step - loss: 6.8722 - accuracy: 0.0589 - val_loss: 7.0050 - val_accuracy: 0.0663
Epoch 6/500
79/79 [==============================] - 9s 115ms/step - loss: 6.8118 - accuracy: 0.0633 - val_loss: 6.9531 - val_accuracy: 0.0651
Epoch 7/500
79/79 [==============================] - 9s 116ms/step 

## V5

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.2))

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model_filename = f'{DATASET}-custom-5.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 500

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

/content/drive/My Drive/Code/autocomplete_me/models/BBC-TECH-custom-5.h5
Epoch 1/500
79/79 [==============================] - 12s 152ms/step - loss: 7.4532 - accuracy: 0.0541 - val_loss: 7.0820 - val_accuracy: 0.0571
Epoch 2/500
79/79 [==============================] - 11s 143ms/step - loss: 7.0215 - accuracy: 0.0578 - val_loss: 7.0999 - val_accuracy: 0.0571
Epoch 3/500
79/79 [==============================] - 11s 142ms/step - loss: 7.0088 - accuracy: 0.0578 - val_loss: 7.1079 - val_accuracy: 0.0571
Epoch 4/500
79/79 [==============================] - 11s 144ms/step - loss: 6.9853 - accuracy: 0.0578 - val_loss: 7.0093 - val_accuracy: 0.0571
Epoch 5/500
79/79 [==============================] - 12s 154ms/step - loss: 6.8651 - accuracy: 0.0621 - val_loss: 6.9215 - val_accuracy: 0.0664
Epoch 6/500
79/79 [==============================] - 12s 146ms/step - loss: 6.7333 - accuracy: 0.0699 - val_loss: 6.8204 - val_accuracy: 0.0715
Epoch 7/500
79/79 [==============================] - 11s 145ms/

## V6

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.5))

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model_filename = f'{DATASET}-custom-6.h5'
# model_filepath = os.path.join(MODELS_DIR, model_filename)
# print(model_filepath)
# # model = load_model(model_filepath)

# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=25),
#     ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
# ]

# EPOCHS = 500

# history = model.fit(
#     X_train, 
#     y_train, 
#     epochs=EPOCHS, 
#     batch_size=2048, 
#     validation_data=(X_test, y_test), 
#     verbose=1,
#     callbacks=callbacks
# )

/content/drive/My Drive/Code/autocomplete_me/models/BBC-TECH-custom-6.h5
Epoch 1/500
79/79 [==============================] - 20s 248ms/step - loss: 7.4586 - accuracy: 0.0542 - val_loss: 7.0811 - val_accuracy: 0.0571
Epoch 2/500
79/79 [==============================] - 18s 223ms/step - loss: 7.0214 - accuracy: 0.0578 - val_loss: 7.1018 - val_accuracy: 0.0571
Epoch 3/500
79/79 [==============================] - 18s 222ms/step - loss: 7.0093 - accuracy: 0.0578 - val_loss: 7.0904 - val_accuracy: 0.0571
Epoch 4/500
79/79 [==============================] - 18s 223ms/step - loss: 6.9560 - accuracy: 0.0578 - val_loss: 7.0012 - val_accuracy: 0.0571
Epoch 5/500
79/79 [==============================] - 18s 233ms/step - loss: 6.8344 - accuracy: 0.0645 - val_loss: 6.8867 - val_accuracy: 0.0687
Epoch 6/500
79/79 [==============================] - 19s 236ms/step - loss: 6.7128 - accuracy: 0.0708 - val_loss: 6.8140 - val_accuracy: 0.0748
Epoch 7/500
79/79 [==============================] - 19s 238ms/

In [ ]:
train_utils.plot_history(history)

# Comparison

### 100 Epochs

<tr>
    <th>LSTM Layers</th>
    <th>LSTM Cells per Layer</th>
    <th>Dropout %</th>
    <th>Validation Loss</th>
    <th>Validation Accuracy</th>
</tr>
<tr>
    <td>1</td>
    <td>64</td>
    <td>0.2</td>
    <td>8.9918</td>
    <td>0.2271</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.2</td>
    <td>10.7950</td>
    <td>0.2354</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.5</td>
    <td>8.9682</td>
    <td>0.2153</td>
</tr>
<tr>
    <td>2</td>
    <td>64</td>
    <td>0.2, 0.5</td>
    <td>6.9549</td>
    <td>0.1490</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.2, 0.5</td>
    <td>7.4581</td>
    <td>0.1683</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.5, 0.5</td>
    <td>7.3286</td>
    <td>0.1650</td>
</tr>

### 500 Epochs

<tr>
    <th>LSTM Layers</th>
    <th>LSTM Cells per Layer</th>
    <th>Dropout %</th>
    <th>Validation Loss</th>
    <th>Validation Accuracy</th>
</tr>
<tr>
    <td>1</td>
    <td>64</td>
    <td>0.2</td>
    <td>16.4351</td>
    <td>0.3229</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.2</td>
    <td>22.5922</td>
    <td>0.3373</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.5</td>
    <td>18.8511</td>
    <td>0.3329</td>
</tr>
<tr>
    <td>2</td>
    <td>64</td>
    <td>0.2, 0.5</td>
    <td>8.8127</td>
    <td>0.2230</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.2, 0.5</td>
    <td>10.8240</td>
    <td>0.2854</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.5, 0.5</td>
    <td>10.4388</td>
    <td>0.2666</td>
</tr>

# Generate Text Data

### Load Objects To Infer

In [7]:
from tensorflow.keras.models import load_model
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-2.h5')
model = load_model(model_filepath)
TRAINING_LENGTH = 10

## Existing Sentences

In [8]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=20,
    diversity=1,
    n_gen=1
)

/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py:42: RuntimeWarning:

divide by zero encountered in log



In [9]:
' '.join(word for word in original_sequence)

'and the current generation of mobiles using flash technology can'

In [10]:
' '.join(word for word in gen_list[0])

'< --- > store data for phones but some readers of the file sharing and both yahoo and bt in europe in the'

In [11]:
' '.join(word for word in a)

'< --- > store up to one gigabyte of music enough for 250 songs we are working in the hard disk area and'

In [28]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=20,
    diversity=1,
    n_gen=1
)

/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py:42: RuntimeWarning:

divide by zero encountered in log



In [29]:
' '.join(word for word in original_sequence)

'cut between 6 000 and 7 000 jobs and reduce'

In [30]:
' '.join(word for word in gen_list[0])

'< --- > costs up from looking for to a international based development in california said iptv was scrapped in japan in 1976'

In [31]:
' '.join(word for word in a)

'< --- > costs by 5bn £2 7bn a year analysts had warned recently that the airline might have to seek chapter 11'

## Custom Sentences

In [ ]:
sentence = 'Stocks of major large technology firms are becoming even more fragile even though'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=20)

[None, 3, 546, 490, 45, 126, 13, 518, 150, 24, 9544, 150, 456]


ValueError: ignored

In [ ]:
sentence = 'However, there have been many instances of'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 56, 18, 46, 67, 7424, 3]


ValueError: ignored